In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [2]:
## Load the training model
model = load_model('model.h5')

## load the encoder and scalar
with open('label_encoder.pkl', 'rb') as file:
    labels_encoder_gender = pickle.load(file)
with open('one_hot_encoder.pkl', 'rb') as file:
    one_hot_encoder = pickle.load(file)
with open('scalar.pkl', 'rb') as file:
    scalar = pickle.load(file)

In [3]:
one_hot_encoder.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [10]:
input_data = {
    'CreditScore': 600,
'Geography': 'France',
'Gender': 'Male',
'Age': 40,
'Tenure': 3,
    'Balance': 600000,
    'NumOfProducts':2,
    "HasCrCard" : 1,
    "IsActiveMember" : 1,
    "EstimatedSalary": 600000,
}

# convert this into dataframe
input_df=pd.DataFrame(input_data,index=[0])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,600000,2,1,1,600000


In [11]:
# One-hot encode Geography
geo_encoded = one_hot_encoder.transform(input_df[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=one_hot_encoder.get_feature_names_out(['Geography']))

# Label encode Gender
input_df['Gender'] = labels_encoder_gender.transform(input_df['Gender'])

In [12]:
input_df = pd.concat([input_df.drop('Geography',axis=1), geo_encoded_df], axis=1)

In [13]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,600000,2,1,1,600000,1.0,0.0,0.0


In [14]:
# scaling
input_scaled =  scalar.transform(input_df)


In [15]:
#predict score
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


array([[0.05221669]], dtype=float32)

In [16]:
prediction_prob = prediction[0][0]

if prediction[0][0] > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
